<a href="https://colab.research.google.com/github/bhargavisaijalluri/ai-travel-planne--using-machine-learning/blob/main/ai_travel_planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import folium
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# ================= LOAD DATA =================
# Make sure this CSV already has 'Latitude' and 'Longitude' columns
df = pd.read_csv("/content/Top Indian Places with Coordinates.csv")

# ================= CLEAN DATA =================
df["Entrance Fee in INR"] = pd.to_numeric(df["Entrance Fee in INR"], errors="coerce").fillna(0)
df["Google review rating"] = pd.to_numeric(df["Google review rating"], errors="coerce").fillna(0)
df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")

# ================= ESTIMATE VISIT TIME =================
def estimate_time(cost):
    if cost == 0:
        return 1.5
    elif cost < 100:
        return 2
    else:
        return 3

df["Estimated Visit Time (hrs)"] = df["Entrance Fee in INR"].apply(estimate_time)

# ================= FEATURE SELECTION =================
features = df[["Entrance Fee in INR", "Google review rating"]]
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# ================= TRAIN ML MODEL =================
kmeans = KMeans(n_clusters=3, random_state=42)
df["Cluster"] = kmeans.fit_predict(scaled_features)

# ================= USER INPUT =================
location = input("Enter City or State Name: ").strip().lower()
days = int(input("Enter Number of Days: "))
total_budget = int(input("Enter Total Budget (₹): "))

# ================= FILTER LOCATION =================
location_data = df[
    df["City"].str.lower().str.contains(location) |
    df["State"].str.lower().str.contains(location)
]

if location_data.empty:
    print("Location not found in dataset.")
else:
    # ================= CLUSTER PREDICTION =================
    avg_budget_per_place = total_budget / days
    user_df = pd.DataFrame([[avg_budget_per_place, 4.0]], columns=["Entrance Fee in INR", "Google review rating"])
    user_scaled = scaler.transform(user_df)
    user_cluster = kmeans.predict(user_scaled)[0]

    cluster_filtered = location_data[location_data["Cluster"] == user_cluster]

    # Relax filter if too few places
    if len(cluster_filtered) < days:
        recommended_places = location_data.sort_values(by="Google review rating", ascending=False)
    else:
        recommended_places = cluster_filtered.sort_values(by="Google review rating", ascending=False)

    print("\n========================================")
    print("        AI TRAVEL PLANNER SYSTEM")
    print("========================================")
    print(f"City              : {location.title()}")
    print(f"Days              : {days}")
    print(f"Daily Time Limit  : 8 Hours")
    print(f"Total Budget      : ₹{total_budget}")
    print("========================================\n")

    current_budget = total_budget
    visited_indices = set()
    skipped_for_map = []

    # ================= DAY-WISE PLANNING =================
    travel_map = folium.Map(location=[recommended_places["Latitude"].mean(),
                                      recommended_places["Longitude"].mean()],
                            zoom_start=6)

    for day in range(1, days + 1):
        day_cost = 0
        day_time = 0
        day_places = []

        for idx, place in recommended_places.iterrows():
            if idx in visited_indices:
                continue

            cost = place["Entrance Fee in INR"]
            visit_time = place["Estimated Visit Time (hrs)"]

            if day_time + visit_time <= 8 and current_budget >= cost:
                day_places.append(place)
                day_cost += cost
                day_time += visit_time
                current_budget -= cost
                visited_indices.add(idx)

                # Add marker if coordinates are valid
                if not pd.isna(place["Latitude"]) and not pd.isna(place["Longitude"]):
                    folium.Marker(
                        location=[place["Latitude"], place["Longitude"]],
                        popup=f"{place['Name']} | ₹{int(cost)} | ⭐{place['Google review rating']}"
                    ).add_to(travel_map)
                else:
                    skipped_for_map.append(place["Name"])

        print(f"Day {day}")
        print("----------------------------------------")

        if not day_places:
            print("No more suitable places available.\n")
            continue

        print(f"Places to Visit : {len(day_places)}")
        print(f"Time Used       : {day_time:.1f} hrs\n")

        for place in day_places:
            print(f"  - {place['Name']}")
            print(f"    ₹{int(place['Entrance Fee in INR'])} | ⭐{place['Google review rating']} | ⏳{place['Estimated Visit Time (hrs)']} hrs\n")

        print(f"Day Cost        : ₹{int(day_cost)}\n")

        if skipped_for_map:
            print(f"Note: Some places were skipped on the map due to missing coordinates:\n{', '.join(skipped_for_map)}\n")
            skipped_for_map = []

    # ================= SUMMARY =================
    used_budget = total_budget - current_budget
    utilization = (used_budget / total_budget) * 100

    print("============== TRIP SUMMARY ==============")
    print(f"Total Used Budget   : ₹{int(used_budget)}")
    print(f"Remaining Budget    : ₹{int(current_budget)}")
    print(f"Budget Utilization  : {utilization:.2f}%")
    print("==========================================\n")

    # ================= SAVE MAP =================
    travel_map.save("travel_plan_map.html")
    print("Map saved as 'travel_plan_map.html'. Open it to see your itinerary locations on the map.")
travel_map

Enter City or State Name: hyderabad
Enter Number of Days: 4
Enter Total Budget (₹): 1500

        AI TRAVEL PLANNER SYSTEM
City              : Hyderabad
Days              : 4
Daily Time Limit  : 8 Hours
Total Budget      : ₹1500

Day 1
----------------------------------------
Places to Visit : 4
Time Used       : 7.5 hrs

  - Charminar
    ₹25 | ⭐4.5 | ⏳2.0 hrs

  - Inorbit Mall Cyberabad
    ₹0 | ⭐4.5 | ⏳1.5 hrs

  - Golconda Fort
    ₹30 | ⭐4.4 | ⏳2.0 hrs

  - Qutb Shahi Tombs
    ₹25 | ⭐4.4 | ⏳2.0 hrs

Day Cost        : ₹80

Day 2
----------------------------------------
Places to Visit : 3
Time Used       : 7.0 hrs

  - Ramoji Film City
    ₹1150 | ⭐4.4 | ⏳3.0 hrs

  - Salar Jung Museum
    ₹20 | ⭐4.4 | ⏳2.0 hrs

  - Chowmahalla Palace
    ₹80 | ⭐4.4 | ⏳2.0 hrs

Day Cost        : ₹1250

Day 3
----------------------------------------
Places to Visit : 3
Time Used       : 5.5 hrs

  - Hussain Sagar Lake
    ₹0 | ⭐4.3 | ⏳1.5 hrs

  - Nehru Zoological Park
    ₹50 | ⭐4.2 | ⏳2.0 hrs

  

In [5]:
travel_map